# Тест Юэна-Уэлча

За реализацию спасибо [mdhaber](https://gist.github.com/mdhaber)

Теорию читать [тут](https://www.ncss.com/wp-content/themes/ncss/pdf/Procedures/PASS/Equivalence_Tests_for_Two_Means-Simulation.pdf),[тут](https://www.jstor.org/stable/pdf/2334299.pdf) и [тут](https://www.jstor.org/stable/pdf/2334550.pdf)


## Функции для trimmed-среднего и winsorized-среднего

**trimmed** - сортируем выборку по возрастанию, откусываем g значений снизу выборки и g значений сверху выборки

**winsorized** - сортируем выборку по возрастанию, первые g значений заменяем на g+1 значение, а последние g значений на N-g'тое значение.

In [5]:
from scipy import stats
import numpy as np


def xbar_tg_fun(x, g):
    n = len(x)
    return 1/(n-2*g)*np.sum(x[g:-g])


def xbar_wg_fun(x, g):
    n = len(x)
    # return 1/n * ((g + 1)*x[g] + np.sum(x[g+1:-g-1]) + (g + 1)*x[-g-1])
    i = np.arange(1, n+1)  # to make indexing really easy
    return 1/n * ((g + 1)*x[i == g+1]
                  + np.sum(x[(i >= g+2) & (i <= n-g-1)])
                  + (g + 1)*x[i == n-g])[0]

## Winsorized сумма ошибок и вспомогательные функции

In [6]:
def SSD_wg_fun(x, g):
    n = len(x)
    xbar_wg = xbar_wg_fun(x, g)
    # return ((g + 1)*(x[g] - xbar_wg)**2
    #         + np.sum((x[g+1:-g-1] - xbar_wg)**2)
    #         + (g + 1)*(x[-g-1] - xbar_wg)**2)
    i = np.arange(1, n + 1)  # to make indexing really easy
    return ((g + 1)*(x[i == g+1] - xbar_wg)**2
            + np.sum((x[(i >= g+2) & (i <= n-g-1)] - xbar_wg)**2)
            + (g + 1)*(x[i == n-g] - xbar_wg)**2)[0]


def s_wg2_fun(x, g):
    h = h_fun(x, g)
    SSD_wg = SSD_wg_fun(x, g)
    return SSD_wg / (h-1)


def h_fun(x, g):
    n = len(x)
    return n - 2*g


def ngh(x, trim):
    n = len(x)
    g = int(trim*n)
    h = h_fun(x, g)
    return n, g, h

## Функции для расчета статистики при равных дисперсиях (и при неравеных дисперсиях)

In [7]:
def t_unequal_var_fun(x1, x2, trim):
    n1, g1, h1 = ngh(x1, trim)
    n2, g2, h2 = ngh(x2, trim)

    x1bar_tg = xbar_tg_fun(x1, g1)
    x2bar_tg = xbar_tg_fun(x2, g2)
    num = (x1bar_tg - x2bar_tg)

    s1_wg2 = s_wg2_fun(x1, g1)
    s2_wg2 = s_wg2_fun(x2, g2)
    den = np.sqrt(s1_wg2 / h1 + s2_wg2 / h2)

    return num/den


def f_unequal_var_fun(x1, x2, trim):
    n1, g1, h1 = ngh(x1, trim)
    n2, g2, h2 = ngh(x2, trim)

    c = c_fun(x1, x2, trim)
    f_recip = c**2/(h1-1) + (1-c)**2/(h2-1)
    return 1/f_recip


def c_fun(x1, x2, trim):
    n1, g1, h1 = ngh(x1, trim)
    n2, g2, h2 = ngh(x2, trim)

    s1_wg2 = s_wg2_fun(x1, g1)
    s2_wg2 = s_wg2_fun(x2, g2)
    return (s1_wg2/h1) / (s1_wg2/h1 + s2_wg2/h2)

def t_equal_var_fun(x1, x2, trim):
    n1, g1, h1 = ngh(x1, trim)
    n2, g2, h2 = ngh(x2, trim)

    x1bar_tg = xbar_tg_fun(x1, g1)
    x2bar_tg = xbar_tg_fun(x2, g2)
    num = (x1bar_tg - x2bar_tg)

    SSD1_wg = SSD_wg_fun(x1, g1)
    SSD2_wg = SSD_wg_fun(x2, g2)
    den = np.sqrt((SSD1_wg + SSD2_wg) / (h1 + h2 - 2) * (1/h1 + 1/h2))

    return num/den


def f_equal_var_fun(x1, x2, trim):
    n1, g1, h1 = ngh(x1, trim)
    n2, g2, h2 = ngh(x2, trim)

    return h1 + h2 - 2


## Ну а теперь завернем все это в одну готовую функцию

In [8]:
def yuen_t_test(x1, x2, trim, equal_var):
    x1 = np.sort(x1)
    x2 = np.sort(x2)
    if equal_var:
        t_fun = t_equal_var_fun
        f_fun = f_equal_var_fun
    else:
        t_fun = t_unequal_var_fun
        f_fun = f_unequal_var_fun
    t = t_fun(x1, x2, trim)
    df = f_fun(x1, x2, trim)
    pval = 2*stats.t.sf(np.abs(t), df)
    return t, pval